In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
## Extracting features of 2022 movies from Wikipedia

In [5]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [6]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [7]:
tables = soup.find_all('table',class_='wikitable sortable')

In [8]:
len(tables)

4

In [9]:
type(tables[0])

bs4.element.Tag

In [10]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [11]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16392\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [12]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[5],NaN
4,JANUARY,14.0,Hotel Transylvania: Transformania,Amazon Studios / Columbia Pictures / Sony Pict...,"Jennifer Kluska, Derek Drymon (directors); Amo...",[6],NaN
...,...,...,...,...,...,...,...
220,DECEMBER,21.0,I Wanna Dance with Somebody,TriStar Pictures / Black Label Media,Kasi Lemmons (director); Anthony McCarten (scr...,NaN,[53]
221,DECEMBER,25.0,Babylon,Paramount Pictures,Damien Chazelle (director/screenplay); Brad Pi...,NaN,[194]
222,DECEMBER,25.0,A Man Called Otto,Columbia Pictures / Playtone / SF Studios,"Marc Forster (director), David Magee (screenpl...",NaN,[53]
223,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_2022 = df[['Title','Cast and crew']]

In [14]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo..."
...,...,...
220,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...
221,Babylon,Damien Chazelle (director/screenplay); Brad Pi...
222,A Man Called Otto,"Marc Forster (director), David Magee (screenpl..."
223,NaN,NaN


In [15]:
!pip install tmdbv3api

In [16]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'ebf9ed9c428df6b662029ca54b5a037d'

In [17]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/550?api_key=ebf9ed9c428df6b662029ca54b5a037d'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [18]:
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

C:\Users\DELL\AppData\Local\Temp\ipykernel_16392\2369738557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))


In [19]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Drama
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Drama
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Drama
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Drama
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Drama
...,...,...,...
220,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama
221,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Drama
222,A Man Called Otto,"Marc Forster (director), David Magee (screenpl...",Drama
223,NaN,NaN,Drama


In [20]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [21]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\DELL\AppData\Local\Temp\ipykernel_16392\3796821595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))


In [22]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [23]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\DELL\AppData\Local\Temp\ipykernel_16392\2929607607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [24]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [25]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\DELL\AppData\Local\Temp\ipykernel_16392\2384233428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [26]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [27]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [28]:
df_2022

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Drama,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Drama,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Drama,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Drama,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Drama,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn
...,...,...,...,...,...,...,...
220,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci
221,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Drama,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire
222,A Man Called Otto,"Marc Forster (director), David Magee (screenpl...",Drama,Marc Forster,Tom Hanks,Manuel Garcia-Rulfo,NaN
223,NaN,NaN,Drama,nan,nan,NaN,NaN


In [29]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [30]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [31]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Drama,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Drama,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Drama,The Commando
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Drama,Scream
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Drama,Hotel Transylvania: Transformania
...,...,...,...,...,...,...
220,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama,I Wanna Dance with Somebody
221,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire,Drama,Babylon
222,Marc Forster,Tom Hanks,Manuel Garcia-Rulfo,NaN,Drama,A Man Called Otto
223,nan,nan,NaN,NaN,Drama,NaN


In [32]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [33]:
new_df22.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      6
actor_3_name     20
genres            0
movie_title       3
comb             20
dtype: int64

In [34]:
new_df22 = new_df22.dropna(how='any')

In [35]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [36]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

C:\Users\DELL\AppData\Local\Temp\ipykernel_16392\2732328091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title'] = new_df22['movie_title'].str.lower()


In [37]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Drama,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Drama,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Drama,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Drama,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Jennife...
5,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug,Drama,shattered,Cameron Monaghan Frank Grillo Lilly Krug Luis ...
...,...,...,...,...,...,...,...
217,James Cameron,Sam Worthington,Zoe Saldana,Sigourney Weaver,Drama,avatar: the way of water,Sam Worthington Zoe Saldana Sigourney Weaver J...
218,David F. Sandberg,Zachary Levi,Jack Dylan Grazer,Rachel Zegler,Drama,shazam! fury of the gods,Zachary Levi Jack Dylan Grazer Rachel Zegler D...
219,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Drama,puss in boots: the last wish,Joel Crawford (director); Antonio Banderas Sal...
220,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama,i wanna dance with somebody,Naomi Ackie Ashton Sanders Stanley Tucci Kasi ...


In [38]:
old_df = pd.read_csv('main2021_data.csv')

In [39]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6452,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Drama,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6453,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
6454,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
6455,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [40]:
final_df = old_df.append(new_df22,ignore_index=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16392\1166909226.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df22,ignore_index=True)


In [41]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6657,James Cameron,Sam Worthington,Zoe Saldana,Sigourney Weaver,Drama,avatar: the way of water,Sam Worthington Zoe Saldana Sigourney Weaver J...
6658,David F. Sandberg,Zachary Levi,Jack Dylan Grazer,Rachel Zegler,Drama,shazam! fury of the gods,Zachary Levi Jack Dylan Grazer Rachel Zegler D...
6659,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Drama,puss in boots: the last wish,Joel Crawford (director); Antonio Banderas Sal...
6660,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama,i wanna dance with somebody,Naomi Ackie Ashton Sanders Stanley Tucci Kasi ...


In [42]:
final_df.to_csv('main2022_data.csv',index=False)